# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 11 2023 8:56AM,254736,10,MSP217430,"Methapharm, Inc.",Released
1,Jan 11 2023 8:56AM,254736,10,MSP217434,"Methapharm, Inc.",Released
2,Jan 11 2023 8:56AM,254736,10,MSP217427,Methapharm-G,Released
3,Jan 11 2023 8:56AM,254736,10,MSP216999,Methapharm-G,Released
4,Jan 11 2023 8:56AM,254736,10,MSP217030,Methapharm-G,Released
5,Jan 11 2023 8:56AM,254736,10,MSP217033,Methapharm-G,Released
6,Jan 11 2023 8:56AM,254736,10,MSP217139,Methapharm-G,Released
7,Jan 11 2023 8:56AM,254736,10,MSP217145,Methapharm-G,Released
8,Jan 11 2023 8:56AM,254736,10,MSP217146,Methapharm-G,Released
9,Jan 11 2023 8:56AM,254736,10,MSP217157,Methapharm-G,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,254732,Released,11
20,254733,Released,1
21,254734,Released,1
22,254735,Released,11
23,254736,Released,10


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254732,NaN,NaN,11.0
254733,NaN,NaN,1.0
254734,NaN,NaN,1.0
254735,NaN,NaN,11.0
254736,NaN,NaN,10.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0.0,0.0,18.0
254655,0.0,1.0,0.0
254657,0.0,0.0,1.0
254668,0.0,1.0,0.0
254673,6.0,5.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254650,0,0,18
254655,0,1,0
254657,0,0,1
254668,0,1,0
254673,6,5,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,0,0,18
1,254655,0,1,0
2,254657,0,0,1
3,254668,0,1,0
4,254673,6,5,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254650,,,18
1,254655,,1,
2,254657,,,1
3,254668,,1,
4,254673,6,5,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc."
2,Jan 11 2023 8:56AM,254736,10,Methapharm-G
10,Jan 11 2023 8:51AM,254735,12,Hush Hush
21,Jan 11 2023 8:47AM,254734,10,Methapharm-G
22,Jan 11 2023 8:45AM,254733,10,Methapharm-G
23,Jan 11 2023 8:41AM,254732,10,ISDIN Corporation
34,Jan 11 2023 8:40AM,254731,19,ACG North America LLC
35,Jan 11 2023 8:30AM,254692,10,ISDIN Corporation
37,Jan 11 2023 8:30AM,254702,10,ISDIN Corporation
39,Jan 11 2023 8:30AM,254710,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc.",,,10
1,Jan 11 2023 8:56AM,254736,10,Methapharm-G,,,10
2,Jan 11 2023 8:51AM,254735,12,Hush Hush,,,11
3,Jan 11 2023 8:47AM,254734,10,Methapharm-G,,,1
4,Jan 11 2023 8:45AM,254733,10,Methapharm-G,,,1
5,Jan 11 2023 8:41AM,254732,10,ISDIN Corporation,,,11
6,Jan 11 2023 8:40AM,254731,19,ACG North America LLC,,,1
7,Jan 11 2023 8:30AM,254692,10,ISDIN Corporation,,,2
8,Jan 11 2023 8:30AM,254702,10,ISDIN Corporation,,,2
9,Jan 11 2023 8:30AM,254710,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc.",10,,
1,Jan 11 2023 8:56AM,254736,10,Methapharm-G,10,,
2,Jan 11 2023 8:51AM,254735,12,Hush Hush,11,,
3,Jan 11 2023 8:47AM,254734,10,Methapharm-G,1,,
4,Jan 11 2023 8:45AM,254733,10,Methapharm-G,1,,
5,Jan 11 2023 8:41AM,254732,10,ISDIN Corporation,11,,
6,Jan 11 2023 8:40AM,254731,19,ACG North America LLC,1,,
7,Jan 11 2023 8:30AM,254692,10,ISDIN Corporation,2,,
8,Jan 11 2023 8:30AM,254702,10,ISDIN Corporation,2,,
9,Jan 11 2023 8:30AM,254710,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc.",10,,
1,Jan 11 2023 8:56AM,254736,10,Methapharm-G,10,,
2,Jan 11 2023 8:51AM,254735,12,Hush Hush,11,,
3,Jan 11 2023 8:47AM,254734,10,Methapharm-G,1,,
4,Jan 11 2023 8:45AM,254733,10,Methapharm-G,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc.",10.0,NaN,NaN
1,Jan 11 2023 8:56AM,254736,10,Methapharm-G,10.0,NaN,NaN
2,Jan 11 2023 8:51AM,254735,12,Hush Hush,11.0,NaN,NaN
3,Jan 11 2023 8:47AM,254734,10,Methapharm-G,1.0,NaN,NaN
4,Jan 11 2023 8:45AM,254733,10,Methapharm-G,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 11 2023 8:56AM,254736,10,"Methapharm, Inc.",10.0,0.0,0.0
1,Jan 11 2023 8:56AM,254736,10,Methapharm-G,10.0,0.0,0.0
2,Jan 11 2023 8:51AM,254735,12,Hush Hush,11.0,0.0,0.0
3,Jan 11 2023 8:47AM,254734,10,Methapharm-G,1.0,0.0,0.0
4,Jan 11 2023 8:45AM,254733,10,Methapharm-G,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2547116,38.0,6.0,6.0
12,254735,11.0,0.0,0.0
15,509348,19.0,0.0,0.0
16,764109,2.0,1.0,0.0
19,1528272,47.0,0.0,0.0
21,509312,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2547116,38.0,6.0,6.0
1,12,254735,11.0,0.0,0.0
2,15,509348,19.0,0.0,0.0
3,16,764109,2.0,1.0,0.0
4,19,1528272,47.0,0.0,0.0
5,21,509312,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,6.0,6.0
1,12,11.0,0.0,0.0
2,15,19.0,0.0,0.0
3,16,2.0,1.0,0.0
4,19,47.0,0.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,12,Released,11.0
2,15,Released,19.0
3,16,Released,2.0
4,19,Released,47.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,0.0
Executing,6.0,0.0,0.0,1.0,0.0,1.0
Released,38.0,11.0,19.0,2.0,47.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,1.0,0.0,1.0
2,Released,38.0,11.0,19.0,2.0,47.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,6.0,0.0,0.0,0.0,0.0,0.0
1,Executing,6.0,0.0,0.0,1.0,0.0,1.0
2,Released,38.0,11.0,19.0,2.0,47.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()